## PyTorch implementation of Deep Bilteral Learning for Real Time Image Enhancement

In [38]:
from __future__ import division, print_function, unicode_literals
import numpy as np
from PIL import Image
import os, sys, glob
# import matplotlib.pyplot as plt
#Torch Imports
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms

In [39]:
size = (256, 256)
batch_size = 100
learning_rate = 0.01
num_epochs = 5

# size of image, right now we are fixing this
xsize = 1920
ysize = 1080

# Constants for slicing layer
sx = 256.0/xsize
sy = 256.0/ysize
d  = 8

# calculating Fx and keeping as constant from precomputation
x = range(xsize)
x = np.asarray(x, dtype=np.float32)
one = np.ones(xsize)
x = np.stack((x, one), axis=1)
x[:, 0] = x[:, 0]*sx
i  = range(16)
i  = np.asarray(i, dtype=np.float32)*-1
one= np.ones(16)
i = np.stack((one, i), axis=1).swapaxes(0, 1)
fx = torch.from_numpy(x)
i  = torch.from_numpy(i)
fxi= torch.matmul(fx, i)
fxi= torch.abs(fxi)
print(fx.shape)
print(fxi.shape)

# calculating Fy and keeping as constant from precomputation
y = range(ysize)
y = np.asarray(y, dtype=np.float32)
one = np.ones(ysize)
y = np.stack((y, one), axis=1)
y[:, 0] = y[:, 0]*sy
y = y.swapaxes(0, 1)
i  = range(16)
i  = np.asarray(i, dtype=np.float32)*-1
one= np.ones(16)
i = np.stack((one, i), axis=1)
fy = torch.from_numpy(y)
i  = torch.from_numpy(i)
fyi= torch.matmul(i, fy)
print(fy.shape)
print(fyi.shape)

torch.Size([1920, 2])
torch.Size([1920, 16])
torch.Size([2, 1080])
torch.Size([16, 1080])


In [40]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, train, transform=None):
        #Init Function
        super(Dataset, self).__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        
        self.full_res = []
        self.low_res = []
        
        if (train):
            dir = self.root_dir + '/train/'
        else :
            dir = self.root_dir + '/test/'
        
        for img_path in glob.glob (dir + '*.jpg'):
            
            himage = Image.open (img_path)
            limage = himage.resize (size)
            
            self.full_res.append (himage)
            self.low_res.append (limage)

    def __len__(self):
        #Length function ?
        return len(self.full_res)

    def __getitem__(self, idx):
        #Accessor Function
        if self.transform is None:
            return (self.full_res[idx],self.low_res[idx])
        else:
            limg_transformed = self.transform(self.low_res[idx])
            himg_transformed =  self.transform(self.full_res[idx])
            return (himg_transformed, limg_transformed)
       

In [41]:
composed_transform = transforms.Compose([transforms.ToTensor()])
train_dataset = Dataset (root_dir = '../data', train = True, transform = composed_transform)
im, im2 = train_dataset.__getitem__(0)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [65]:
class LocalFeatureNet (nn.Module):
           
    def __init__(self):
        super (LocalFeatureNet, self).__init__()
        
        self.relu  = nn.ReLU (inplace = True)
        
        self.conv1 = nn.Conv2d (in_channels = 3,   out_channels = 8, kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d (in_channels = 8,  out_channels = 16,  kernel_size = 3, stride = 2, padding = 1)
        self.conv3 = nn.Conv2d (in_channels = 16, out_channels = 32,  kernel_size= 3, stride = 2,padding = 1)
        self.conv4 = nn.Conv2d (in_channels = 32, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        
        self.localconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 1, padding = 1)
        self.localconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size= 3, stride = 1, padding = 1)
        
        self.globalconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2,padding = 1)
        self.globalconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        self.globalfc1 = nn.Linear (1024, 256)
        self.globalfc2 = nn.Linear (256, 128)
        self.globalfc3 = nn.Linear (128, 64)
        self.linear = nn.Conv2d(in_channels = 64, out_channels = 96,  kernel_size = 1, stride = 1)
        
        # Pixel Wise Network
        self.pixelwise_bias = nn.Parameter (torch.rand(3, 1), requires_grad=True)
        self.pixelwise_weight = nn.Parameter (torch.eye(3), requires_grad = True)
        self.pixelwise_obias = nn.Parameter (torch.eye(1), requires_grad = True)
        
        self.relu_slopes = nn.Parameter(torch.rand(3,16), requires_grad = True)
        self.relu_shifts = nn.Parameter(torch.rand(16,3), requires_grad = True)
        
        # Thing we do for upsampling
        self.Fx = fxi
        self.Fy = fyi
        
        
    def custom_relu(self,channel,value):
        print("value",value.size())
        size = (16,value.size()[0],value.size()[1],value.size()[2])
        size_alt = (1L,value.size()[0],value.size()[1],value.size()[2])
        print("size: ",size,"size_alt",size_alt)
        value = value.expand(size)
        
        print(self.relu_shifts[:,channel])
        a = self.relu_shifts[:,channel].clone()
        a = a.view(16,1,1,1)
#         print(a.data,a)
#         value = self.relu(value - a.repeat(size_alt))    
# Upper line is not working for some damn reason so I hard coded it for now.
#         print ("here ", a.repeat(1,1,1080,1920).size())
        value = self.relu(value - a.repeat(1,1,1080,1920))
        print("st ", value.size())
        print ("st2 ", self.relu_slopes.size())
        value = self.relu_slopes.matmul(value.view(1080,1920,16,1))
        print ("stt ", value.size())
        value = value.view(1,3,1080,1920)
        value = value[:,1,:,:]+value[:,2,:,:]+value[:,0,:,:]
        print ("stt2 ", value.size())
        return value
    
    def upsample(self, grid, bilat):
        
        G = grid.clone()
        A = bilat.clone()
        A = A.squeeze(0) # why is this worthless ?
        print("------------------------------------")
        print(G.size())
        print("------------------------------------")
        
        
    def output(self, grid, inp):
        out = torch.rand(1,1080,1920,3)
        out = 0 * out
        grid = grid.view(-1,1080,1920,12)
        
        for i in range(0,3):
            out[:,:,:,i] = out[:,:,:,i] + grid[:,:,:,3+4*i]
            for j in range(0,3):
                print(grid[:,:,:,j+4*i].size())
                print(inp[:,j,:,:].size())
                a = torch.autograd.Variable(grid[:,:,:,j+4*i], requires_grad=False)
#                 a = a.numpy()
                b = inp[:,j,:,:]
#                 b = b.numpy()
                temp = a.data() * b.data()
                print(temp.size())
                out[:,:,:,i] = out[:,:,:,i] + temp
        
        return out
    
    def forward (self, h, l):
        
#         print (x.size())
        
        x = self.relu (self.conv1 (l))
#         print (x.size())
        x = self.relu (self.conv2 (x))
#         print (x.size())
        x = self.relu (self.conv3 (x))
#         print (x.size())
        x = self.relu (self.conv4 (x))
#         print (x.size())
#         print ("lel")
        y = self.localconv1 (x)
#         print (y.size())
        y = self.localconv2 (y)
        
        print (y.size())
        
        z = self.globalconv1 (x)
        z = self.globalconv2 (z)
        z = self.globalfc1 (z.view(1, -1))
        z = self.globalfc2 (z)
        z = self.globalfc3 (z)
        
        print (z.size())
        
        
        fused = self.relu(z.view(-1,64,1,1)+y)
        print (fused.size())
        
        lin = self.linear(fused)
        print (lin.size())
        
        bilat = lin.view(-1, 8, 3, 4, 16, 16)
        
        
#         bilat = self.bilateralGrid (fused)
        
        print (bilat.size())
        
#         return bilat

# pixel wise network
        for i in range(0,3):
            a = self.pixelwise_weight[i,:].view(1,3)
            b = h.unsqueeze(0).view(3,-1)
            print(a.size(),b.size())
            p = torch.mm(a,b)
            print(p.size())
            p = p.view(h.size()[0],h.size()[2],h.size()[3]) + self.pixelwise_bias[i]
            print(p.size())
#             p = torch.bmm(self.pixelwise_weight[i,:], h.unsqueeze(0).view(3,-1)).view(h.size()) + self.pixelwise_bias[i]
            print(i)
            p += self.custom_relu(i,p)
        p += self.pixelwise_obias
        print(p.size())
        grid = p
        print("+++++++++++++++++++")
        print(bilat.size())
        print("+++++++++++++++++++")
        bilat_new = self.upsample(grid, bilat)
        
        return self.output(bilat_new,h)
        
        

In [60]:
model = LocalFeatureNet() 

# Add code for using CUDA here if it is available
use_gpu = False
# if(torch.cuda.is_available()):
#     use_gpu = True
#     model.cuda()

# Loss function and optimizers
criterion = torch.nn.MSELoss()# Define MSE loss
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)# Use Adam optimizer, use learning_rate hyper parameter

In [61]:
def train():
    # Code for training the model
    # Make sure to output a matplotlib graph of training losses
    loss_arr = []
    for epoch in range(num_epochs):
        for i, (himage, limage) in enumerate(train_loader):  
            # Convert torch tensor to Variable
            himage = Variable(himage)
            limage = Variable(limage)
            if(use_gpu):
                himage=himage.cuda()
                limage=limage.cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = model(himage, limage)
#             loss = criterion(outputs, input)
#             loss.backward()
#             optimizer.step()
#             loss_arr.append(loss.data[0])
#             if (i+1) % batch_size == 0:       
#                 print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
#                        %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    
#     plt.plot( np.array(range(1,len(loss_arr)+1)), np.array(loss_arr))
#     plt.show()


In [66]:
%time train()

torch.Size([1, 64, 16, 16])
torch.Size([1, 64])
torch.Size([1, 64, 16, 16])
torch.Size([1, 96, 16, 16])
torch.Size([1, 8, 3, 4, 16, 16])
torch.Size([1, 3]) torch.Size([3, 2073600])
torch.Size([1, 2073600])
torch.Size([1, 1080, 1920])
0
value torch.Size([1, 1080, 1920])
size:  (16, 1L, 1080L, 1920L) size_alt (1L, 1L, 1080L, 1920L)
Variable containing:
 0.5026
 0.0820
 0.3103
 0.0941
 0.9054
 0.7577
 0.2378
 0.8744
 0.6584
 0.7551
 0.0847
 0.2877
 0.9287
 0.5638
 0.9624
 0.2930
[torch.FloatTensor of size 16]

st  torch.Size([16, 1, 1080, 1920])
st2  torch.Size([3, 16])
stt  torch.Size([1080, 1920, 3, 1])
stt2  torch.Size([1, 1080, 1920])
torch.Size([1, 3]) torch.Size([3, 2073600])
torch.Size([1, 2073600])
torch.Size([1, 1080, 1920])
1
value torch.Size([1, 1080, 1920])
size:  (16, 1L, 1080L, 1920L) size_alt (1L, 1L, 1080L, 1920L)
Variable containing:
 0.2493
 0.1212
 0.7170
 0.0594
 0.3754
 0.7662
 0.7641
 0.6001
 0.1154
 0.8196
 0.0073
 0.4610
 0.7466
 0.4816
 0.6935
 0.2969
[torch.Float

AttributeError: 'NoneType' object has no attribute 'view'